# load library

In [1]:
source("r/jupyter_common.R")
source("r/identify_cell_types.R")
source("r/plot_sc_clusters.R")

source("r/archr/archr_functions_modified.R")



In [2]:
# data structure
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(GenomicRanges))

# single cell analysis
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(ArchR))


In [3]:
addArchRThreads(threads=1)
getArchRThreads()

addArchRGenome("hg38")


Setting default number of Parallel threads to 1.



[1] 1

Setting default genome to Hg38.



# parameters

In [4]:
str_condition <- "male-bc"

args <- list()
args$cancer_type <- "male-bc"
args$method_to_identify_cell_types <- "singler_blueprint_encode"
args$harmony_theta <- 0

str_column_of_meta_data_harmony <- sprintf("RNA_harmony_th.%s", paste(args$harmony_theta, collapse=","))

str_reduction <- "pca"
str_umap_reduction <- "umap"  
col_cell.type <- "cell.type"

if (args$harmony_theta >= 0) {
    str_column_of_meta_data_cluster <- str_column_of_meta_data_harmony
    str_reduction <- "harmony"
    str_umap_reduction <- "umapharmony"
    col_cell.type <- "cell.type.harmony"
}


str_column_of_meta_data_cluster
str_umap_reduction
col_cell.type


figure_format <- "pdf"


[1] "RNA_harmony_th.0"

[1] "umapharmony"

[1] "cell.type.harmony"

## set.seed

In [5]:
# set seed for reproducibility
set.seed(51)



## display

In [6]:
options(repr.matrix.max.cols=150, repr.matrix.max.rows=50)



In [7]:
addArchRVerbose(verbose = FALSE)


Setting addArchRVerbose = FALSE



# read peaks

## gr_encode

In [8]:
df_encode.all <- read.delim("./reference/genome_annotation/GRCh38-cCREs.bed.gz", header =F)
colnames(df_encode.all)[1:3] <- c("seqnames","start","end")

gr_encode <<- makeGRangesFromDataFrame(df_encode.all)



## gr_hmec

In [9]:
gr_hmec <- readRDS("reference/normal_cell_lines_breast/HMEC_H3K27ac_peaks.rds")


# read scATAC-seq

In [10]:
dir_atac <- "/home/hkim77/francolab.w/sc-atac-seq/male-bc/run-20220725"
dir_output <- sprintf("%s/output_male-bc", dir_atac)
dir_output_p2g <- sprintf("%s/output_p2g_male-bc", dir_atac)
fname_atac <- sprintf("%s/rds/male-bc_archrproj_obj_final.rds", dir_output)

atac <- readRDS(fname_atac)
atac



           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /datastore/nextgenout5/share/labs/francolab/hyunsoo.kim/sc-atac-seq/male-bc/run-20220725/output_male-bc/archr_output 
samples(2): 4CC61L 446B7L
sampleColData names(1): ArrowFiles
cellColData names(47): Sample TSSEnrichment ... RNA_harmony_th.0
  cluster.type.harmony
numberOfCells(1): 10379
medianTSS(1): 10.012
medianFrags(1): 6247

## list_sort_atac

In [11]:

list_sort_atac <- sort_cluster_members(atac, args,
                    col_cluster_types = col_cluster_types,
                    str_umap_reduction = str_umap_reduction,
                    f_merge_immune_cell = FALSE)



Sample,Endothelial cells,Epi. Tumor,Epi. Unassigned,Fibroblasts,Macrophages
<chr>,<int>,<int>,<int>,<int>,<int>
446B7L,2204,1624,1027,1622,183
4CC61L,NA,2629,NA,404,686


Sample
<chr>
446B7L
4CC61L


Sample,Endothelial cells,Epi. Tumor,Epi. Unassigned,Fibroblasts,Macrophages
<chr>,<int>,<int>,<int>,<int>,<int>
446B7L,2204,1624,1027,1622,183
4CC61L,NA,2629,NA,404,686


# read proj.archr.for_comparison

In [12]:
fname_atac.for_comparison <- sprintf("%s/rds/proj.archr.for_comparison.normal-vs-cancer_cancer-specific_enhancer.rds",
                        dir_output_p2g)

atac.for_comparison <- readRDS(fname_atac.for_comparison)
atac.for_comparison



           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /datastore/nextgenout5/share/labs/francolab/hyunsoo.kim/sc-atac-seq/male-bc/run-20220725/output_p2g_male-bc/archr_output 
samples(2): 446B7L 4CC61L
sampleColData names(1): ArrowFiles
cellColData names(47): Sample TSSEnrichment ... RNA_harmony_th.0
  cluster.type.harmony
numberOfCells(1): 9100
medianTSS(1): 9.99
medianFrags(1): 6419.5

# overlap with p2g.df.sub.plot.cancer_specific

## load p2g.df.sub.plot.cancer_specific

In [13]:
peaktype <- "enhancer"
fname_rds <- sprintf("%s/rds/cancer_specific_%s_p2g_table.rds", dir_output_p2g, peaktype)

p2g.df.sub.plot.cancer_specific <- readRDS(fname_rds)
head(p2g.df.sub.plot.cancer_specific)
dim(p2g.df.sub.plot.cancer_specific)

n_p2g.cancer_specific <- nrow(p2g.df.sub.plot.cancer_specific)
var <- sprintf("n_p2g.%s.cancer_specific", peaktype)
cat(sprintf("\t%s: %d\n", var, n_p2g.cancer_specific))

n_peaks.p2g.cancer_specific <- length(unique(p2g.df.sub.plot.cancer_specific$idxATAC))
var <- sprintf("n_peaks.p2g.%s.cancer_specific", peaktype)
cat(sprintf("\t%s: %d\n", var, n_peaks.p2g.cancer_specific))


,idxATAC,idxRNA,Correlation,FDR,VarQATAC,VarQRNA,EmpPval,EmpFDR,geneName,peakName,peakType,idx,kmeans
,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
1,29,3,0.5394961,6.024423e-38,0.9396097,0.4338114,0.11278982,0.7965846,FAM41C,chr1:1079324-1079824,Distal,29-3,5
2,24,7,0.7894052,5.401495e-105,0.7539227,0.5385226,0.01994288,0.7892789,PLEKHN1,chr1:1068557-1069057,Distal,24-7,5
3,25,7,0.7349978,6.699672e-84,0.7787057,0.5385226,0.03031435,0.7892789,PLEKHN1,chr1:1069078-1069578,Distal,25-7,2
4,26,7,0.8462787,5.132060e-135,0.7508486,0.5385226,0.01255312,0.7892789,PLEKHN1,chr1:1069752-1070252,Distal,26-7,2
5,29,7,0.8539726,5.278697e-140,0.9396097,0.5385226,0.01176771,0.7892789,PLEKHN1,chr1:1079324-1079824,Distal,29-7,5
6,30,7,0.6868139,3.617362e-69,0.4759030,0.5385226,0.04307672,0.7892789,PLEKHN1,chr1:1080052-1080552,Distal,30-7,5


[1] 11551    13

	n_p2g.enhancer.cancer_specific: 11551
	n_peaks.p2g.enhancer.cancer_specific: 5141


## df_dge.cse

In [14]:

filename <- "tsv/df_dge.cse.tsv"
df_dge.cse <- read.table(filename, sep='\t', header=T)

head(df_dge.cse)
dim(df_dge.cse)



,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,expr_min,expr_mean,expr_max
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ISG20,3.978188e-146,2.525560,0.945,0.118,9.958995e-142,0.0000000,1.751471,3.843429
S100A6,2.320176e-145,3.401509,1.000,0.574,5.808329e-141,0.0000000,4.662926,6.206801
SAT1,8.599381e-139,2.803190,1.000,0.667,2.152769e-134,0.2981702,3.742109,5.598026
MYL12A,1.249291e-130,2.150761,0.998,0.570,3.127474e-126,0.0000000,2.650992,4.167212
PPP1R14B,8.975878e-129,1.897139,0.995,0.375,2.247021e-124,0.0000000,2.040378,3.449861
ANXA2,7.186044e-128,1.925806,0.999,0.668,1.798954e-123,0.0000000,2.983556,4.697529


[1] 363   8

## p2g_genes_female_vs_male.cse

In [15]:
f <- (abs(df_dge.cse$avg_log2FC) > 1.0 & df_dge.cse$p_val_adj < 0.01)
p2g_genes_female_vs_male.cse <- rownames(df_dge.cse[f,])
p2g_genes_female_vs_male.cse
print(length(p2g_genes_female_vs_male.cse))

if (length(p2g_genes_female_vs_male.cse) < 5) {
    p2g_genes_female_vs_male.cse <- rownames(df_dge.cse)
    print(p2g_genes_female_vs_male.cse)
    print(length(p2g_genes_female_vs_male.cse))
}


[1] "ISG20"    "S100A6"   "SAT1"     "MYL12A"   "PPP1R14B" "ANXA2"   
 [7] "MYL12B"   "TKT"      "HSPB8"    "S100A16"  "PIM3"     "NAA20"   
[13] "SH3BGRL3" "SQOR"     "FHL2"     "LAMB3"    "CAPN2"    "TALDO1"  
[19] "CLTB"     "PFKP"     "USP53"    "PHLDA2"   "SLC39A1"  "CD47"    
[25] "NDUFV2"   "PAWR"     "PPIF"     "BAG1"     "CSRP1"    "SLK"     
[31] "CLIP1"    "MYH9"     "C1QBP"    "MGLL"     "ZNF706"   "PHLDA3"  
[37] "PRDX4"    "S100A14"  "HES4"     "GNAS"     "SFN"      "VDAC2"   
[43] "GTF2A2"   "TMEM40"   "MGST1"    "FABP3"    "CCT3"     "FSTL3"   
[49] "METRN"    "TUBB4B"   "ACTN1"    "TACSTD2"  "FBXO32"   "SPTSSA"  
[55] "KRT23"    "GDF15"    "SERPINA1" "WFDC2"    "YWHAZ"    "KRT7"    
[61] "TPM1"

[1] 61


# overlap with markerpeaks

## read markerpeaks.for_comparion

In [16]:
fname_rds <- sprintf("%s/rds/markerpeaks.for_comparison.normal-vs-cancer_cancer-specific_enhancer.rds",
                     dir_output_p2g)

markerpeaks.for_comparison <- readRDS(fname_rds)
markerpeaks.for_comparison


class: SummarizedExperiment 
dim: 10070 1 
metadata(2): MatchInfo Params
assays(7): Log2FC Mean ... AUC MeanBGD
rownames(10070): 11 22 ... 99784 99839
rowData names(4): seqnames idx start end
colnames(1): Cancer
colData names(0):

## df_markpeaks

In [17]:
list.markerpeaks <- getMarkers(markerpeaks.for_comparison,
                         cutOff = "FDR <= 0.1 & Log2FC >= 1.0")

df_markpeaks <- as.data.frame(list.markerpeaks[[1]])
df_markpeaks$peakName <- sprintf("%s:%d-%d", df_markpeaks$seqnames, df_markpeaks$start, df_markpeaks$end)

# cancer_specific
p2g.df.tmp <- p2g.df.sub.plot.cancer_specific

idx <- match(df_markpeaks$peakName, p2g.df.tmp$peakName)


f.na <- is.na(idx)
any(f.na)
# FALSE: all rows have peak2gene links

f <- !f.na; idx <- idx[f]
df_markpeaks[f, "geneName"] <- p2g.df.tmp[idx, "geneName"]


head(df_markpeaks)
dim(df_markpeaks)


[1] TRUE

,seqnames,idx,start,end,Log2FC,FDR,MeanDiff,peakName,geneName
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
91628,chr8,3644,101360826,101361326,3.036088,1.265616e-15,0.7695004,chr8:101360826-101361326,ZNF706
6866,chr1,6866,172499556,172500056,4.021133,2.145851e-15,0.7574420,chr1:172499556-172500056,PIGC
89631,chr8,1647,37903898,37904398,3.512107,2.422439e-15,0.5867696,chr8:37903898-37904398,ZNF703
92418,chr8,4434,123596296,123596796,3.038512,5.364745e-15,0.5969846,chr8:123596296-123596796,WDYHV1
47852,chr19,2475,40921046,40921546,3.581508,2.871754e-14,0.6185288,chr19:40921046-40921546,ITPKC
90801,chr8,2817,74073118,74073618,5.214121,2.871754e-14,0.6343315,chr8:74073118-74073618,UBE2W


[1] 3998    9

# peak2gene links

In [18]:

marker_genes <- p2g_genes_female_vs_male.cse

marker_genes <- sort(marker_genes)
marker_genes
length(marker_genes)


[1] "ACTN1"    "ANXA2"    "BAG1"     "C1QBP"    "CAPN2"    "CCT3"    
 [7] "CD47"     "CLIP1"    "CLTB"     "CSRP1"    "FABP3"    "FBXO32"  
[13] "FHL2"     "FSTL3"    "GDF15"    "GNAS"     "GTF2A2"   "HES4"    
[19] "HSPB8"    "ISG20"    "KRT23"    "KRT7"     "LAMB3"    "METRN"   
[25] "MGLL"     "MGST1"    "MYH9"     "MYL12A"   "MYL12B"   "NAA20"   
[31] "NDUFV2"   "PAWR"     "PFKP"     "PHLDA2"   "PHLDA3"   "PIM3"    
[37] "PPIF"     "PPP1R14B" "PRDX4"    "S100A14"  "S100A16"  "S100A6"  
[43] "SAT1"     "SERPINA1" "SFN"      "SH3BGRL3" "SLC39A1"  "SLK"     
[49] "SPTSSA"   "SQOR"     "TACSTD2"  "TALDO1"   "TKT"      "TMEM40"  
[55] "TPM1"     "TUBB4B"   "USP53"    "VDAC2"    "WFDC2"    "YWHAZ"   
[61] "ZNF706"

[1] 61

## plotPDF

In [19]:

grangeslist_features <- GRangesList(Peaks = getPeakSet(atac),
                                HMEC = gr_hmec,
                                Encode = gr_encode)


### usegroups

In [20]:
peakset.for_comparison <- getPeakSet(atac.for_comparison)
usegroups <- unique(names(peakset.for_comparison))
f <- !grepl("Unassigned|6-", usegroups)
usegroups <- usegroups[f]
usegroups


[1] "0-Epi. Tumor"        "2-Epi. Tumor"        "3-Endothelial cells"
[4] "4-Fibroblasts"       "7-Macrophages"

In [21]:
library(conflicted)
conflicted::conflict_prefer("%over%", "IRanges")


[conflicted] Will prefer IRanges::`%over%` over any other package


### marker_genes

In [22]:
marker_genes <- c("ANXA2", "LAMB3", "PRDX4")


### plot_browser_track

In [23]:

str_upstream_downstream <- "LAMB3=150000,10000"
out <- plot_browser_track(atac,
                atac.for_comparison,
                df_markpeaks,          
                marker_genes,
                list_sort_atac,
                grangeslist_features,
                str_upstream_downstream,
                usegroups=usegroups,
                f_debug=FALSE)


archr_peak2gene_ANXA2.pdf

archr_peak2gene_LAMB3.pdf

archr_peak2gene_PRDX4.pdf

# session info

In [24]:
writeLines(capture.output(sessionInfo()), "txt/sessionInfo.txt")


# reference

https://satijalab.org/seurat/

